In [1]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
import tensorflow
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Build a Neural Network
This convolution network consists of two pairs of Conv and MaxPool layers to extract features from the dataset. Which is then followed by a Flatten and Dropout layer to convert the data in 1D and ensure overfitting.

And then two Dense layers for classification.

In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation = 'relu', input_shape = (150,150,3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation = 'relu'),
    Dense(2, activation = 'softmax')
])
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['acc'])

#  Image Data Generation/Augmentation

In [3]:
## ImageDataGenerator class => Generate batches of tensor image data with real-time data augmentation.
## Rescale 1./255 is to transform every pixel value from range [0,255] -> [0,1]. And the benefits are: 
# Treat all images in the same manner: some images are high pixel range, some are low pixel range.

train_dir = "C:\\Users\\gkhat\\Desktop\\Dataset\\train"
train_datagen = ImageDataGenerator (rescale = 1.0/255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 10,
                                                    target_size = (150,150))

validation_dir = "C:\\Users\\gkhat\\Desktop\\Dataset\\test"
validation_datagen = ImageDataGenerator(rescale = 1.0/255)

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              batch_size = 10,
                                                              target_size = (150,150))

Found 1275 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


## Initialize a callback checkpoint to keep saving best model after each epoch while training

In [4]:
checkpoint = ModelCheckpoint('model12-{epoch:03d}.model',
                            monitor = 'val_loss',
                            verbose = 0,
                            save_best_only = True,
                            mode = 'auto')

# Train the model

In [5]:
import os
from PIL import Image
folder_path = r'C:\Users\gkhat\Desktop\Dataset\train'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

In [6]:
train = model.fit(train_generator,
                           epochs = 10,
                           validation_data = validation_generator,
                           callbacks = [checkpoint])

Epoch 1/10
128/128 [==============================] - ETA: 0s - loss: 0.6179 - acc: 0.7271WARNING:tensorflow:From C:\Users\gkhat\anaconda3\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model12-001.model\assets
128/128 [==============================] - 108s 844ms/step - loss: 0.6179 - acc: 0.7271 - val_loss: 0.2728 - val_acc: 0.9278
Epoch 2/10
128/128 [==============================] - 119s 929ms/step - loss: 0.2749 - acc: 0.9035 - val_loss: 0.1181 - val_acc: 0.9536
Epoch 3/10
128/128 [==============================] - 117s 916ms/step - loss: 0.1843 - acc: 0.9396 - val_loss: 0.